## This notebook converts downloaded HTML data from Scottish Assessors into a .CSV file (via a Pandas table)

In [2]:
!pip install convertbng
from convertbng.util import convert_bng, convert_lonlat
import json
import pandas as pd
import geopandas
from shapely.geometry import Polygon, LineString, Point
import os



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "C:\ProgramData\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "C:\ProgramData\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "C:\ProgramData\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
    self.io_loop.start()
  File "C:\ProgramData\anaconda3\Lib\site-packa

AttributeError: _ARRAY_API not found

# Get POIs from assessors data 

Save the.HTML files from the Scottish Assesors site.

These files contain details of non-residential buildings based in tax (business rates) data.
We will use this to generates places of interest that will define how useful a bus stop is


In [3]:
# Useful functions to convert from Eastings & Northings to Lat Lon
# SA data uses Eastings & Northings internally - convert to Lat Lon for compatability

def convertToLat(east,north):
    return convert_lonlat([east], [north])[1][0]

def convertToLon(east,north):
    return convert_lonlat([east], [north])[0][0]


In [24]:
# Convert HTML to Pandas (via JSON)

def importSAFile(path,fileName):
#     path to the data dir and file being the .HTML file
    # Open the file in read mode
    with open(str(path)+str(fileName), 'r',encoding = "ISO-8859-1") as file:
        # Read each line in the file
        for line in file:
            # Print each line
            if 'var markers = ' in line:
              jsonStr = line.strip()
              break

        jsonStr = jsonStr[14:]
        jsonStr = jsonStr[:-1]

        # print(jsonStr)


    with open(str(path)+str(fileName)+'.json', 'w') as json_file:
        json.dump(jsonStr, json_file, indent=4)

    with open(str(path)+str(fileName)+'.json','r') as f:
      data = json.load(f)

#     Assume that the filename contains the category

    category = fileName.replace('.html','')
    category = fileName.replace('.htm','')
        
    table = pd.read_json(data)
    table['category'] = category.split('-')[0]
#     Add postcode
    table['postcode'] = ''
    
    def getPCode(row):
        pc = row['name']
        pc = pc[-7:]
        row['postcode'] = pc
        return row
    
    table = table.apply(getPCode, axis=1)    
    return table

# importSAFile('./dundee/','Industrial-DD1.htm')

In [25]:
def getFiles(folder):
#     return a list of .HTML files in <folder>
    files = []
    for x in os.listdir(folder):
        if x.endswith(".html") or x.endswith(".htm"):
            # Prints only text file present in My Folder
            files.append(x)
    return files

getFiles('./dundee/')

['CareFacilities.htm',
 'Cultural.htm',
 'EducationAndTraining.htm',
 'GaragesAndPetrol.htm',
 'Health.htm',
 'Hotels.htm',
 'Industrial-DD1.htm',
 'Industrial-DD2.htm',
 'Industrial-DD3.htm',
 'Industrial-DD4.htm',
 'Industrial-DD5.htm',
 'Leisure.htm',
 'Offices-DD1.htm',
 'Offices-DD2.htm',
 'Offices-DD3.htm',
 'Offices-DD4.htm',
 'Offices-DD5.htm',
 'other.htm',
 'PetroChemical.htm',
 'PublicService.htm',
 'Pubs.htm',
 'QuarriesMines.htm',
 'Religious.htm',
 'Shops-DD1.htm',
 'Shops-DD2.htm',
 'Shops-DD3.htm',
 'Shops-DD4.htm',
 'Shops-DD5.htm',
 'Sporting.htm',
 'Undertaking.htm']

In [26]:
# read in POIS 
folder = './dundee/'

pois = pd.DataFrame()
files = getFiles(folder)
for f in files:
    pois = pois._append(importSAFile(folder,f), ignore_index=True)

# Add lat lon to each pois
pois['lattitude'] = pois.apply(lambda x: convertToLat(x['east'], x['north']), axis=1)
pois['longitude'] = pois.apply(lambda x: convertToLon(x['east'], x['north']), axis=1)

pois.to_csv(folder + 'pois.csv')
gPois = geopandas.GeoDataFrame(pois, geometry=geopandas.points_from_xy(pois.longitude, pois.lattitude), crs="EPSG:4326")

print(gPois)

C:\Users\40000408\AppData\Local\Temp\ipykernel_17268\1606373574.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_json(data)
C:\Users\40000408\AppData\Local\Temp\ipykernel_17268\1606373574.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_json(data)
C:\Users\40000408\AppData\Local\Temp\ipykernel_17268\1606373574.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_json(data)
C:\Users\40000408\AppData\Local\Temp\ipykernel_17268\1606373574.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a l

                                        prop  \
0                        DUNDEE CITY COUNCIL   
1          MICHAEL J & MRS ELEANOR H MAHONEY   
2                    THE RICHMOND FELLOWSHIP   
3                        DUNDEE CITY COUNCIL   
4                                JACK VISSER   
...                                      ...   
6533                         JOHN R.S. GRANT   
6534                    FAIRLIE BIOFUELS LTD   
6535                        A G M FORBES LTD   
6536  SEAGREEN PHASE 1 OFTO PROJECT  LIMITED   
6537   PORT OF DUNDEE LTD DUNDEE PORT OFFICE   

                                                   name  \
0     JANET BROUGHAM HOUSE, 1 BANCHORY ROAD, DUNDEE,...   
1       DUDHOPE VILLA, 1 ST MARY PLACE, DUNDEE, DD1 5RB   
2             FLAT 8, 2 PITAIRLIE ROAD, DUNDEE, DD4 8DB   
3                   3 KINGS CROSS ROAD, DUNDEE, DD2 3PT   
4       3 ARGYLE PLACE, THOMSON STREET, DUNDEE, DD1 4LE   
...                                                 ...   
6533  WEST